# TensorFlow and Keras Deep Learning Basics

This notebook demonstrates fundamental operations with TensorFlow and Keras.

**Libraries:**
- [TensorFlow](https://tensorflow.org/) - End-to-end ML platform
- [Keras](https://keras.io/) - High-level neural networks API

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Suppress TF warnings

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.20.0
Keras version: 3.13.1


## TensorFlow Basics

TensorFlow tensors are similar to NumPy arrays but optimized for GPU acceleration.

In [ ]:
# Check GPU availability
gpus = tf.config.list_physical_devices("GPU")
print(f"GPUs available: {len(gpus)}")
if gpus:
    for gpu in gpus:
        print(f"  {gpu}")

In [ ]:
# Creating tensors
x = tf.constant([1, 2, 3, 4, 5], dtype=tf.float32)
y = tf.random.normal((3, 4))
z = tf.zeros((2, 3))
ones = tf.ones((2, 2))

print(f"1D Tensor: {x.numpy()}")
print(f"Random tensor shape: {y.shape}")
print(f"Zeros tensor:\n{z.numpy()}")

In [ ]:
# Tensor operations
a = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
b = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

print(f"Matrix A:\n{a.numpy()}")
print(f"Matrix B:\n{b.numpy()}")
print(f"\nMatrix multiplication (A @ B):\n{tf.matmul(a, b).numpy()}")
print(f"\nElement-wise multiplication (A * B):\n{(a * b).numpy()}")
print(f"\nSum: {tf.reduce_sum(a).numpy()}")
print(f"Mean: {tf.reduce_mean(a).numpy()}")

## Automatic Differentiation with GradientTape

TensorFlow uses `GradientTape` to record operations for automatic differentiation.

In [ ]:
x = tf.Variable([2.0, 3.0])

with tf.GradientTape() as tape:
    y = x ** 2 + 3 * x + 1

gradients = tape.gradient(y, x)

print(f"x = {x.numpy()}")
print(f"y = x^2 + 3x + 1 = {y.numpy()}")
print(f"dy/dx = 2x + 3 = {gradients.numpy()}")
print(f"\nVerification: 2*[2,3] + 3 = [{2*2+3}, {2*3+3}]")

## Keras Sequential Model

The Sequential API is the simplest way to build neural networks in Keras.

In [ ]:
# Create a simple sequential model
model = models.Sequential([
    layers.Dense(64, activation="relu", input_shape=(10,)),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(2, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

## Keras Functional API

The Functional API provides more flexibility for complex architectures.

In [ ]:
# Define model using Functional API
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

functional_model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

print(f"Model name: {functional_model.name}")
print(f"Input shape: {functional_model.input_shape}")
print(f"Output shape: {functional_model.output_shape}")

## Training Example (XOR Problem)

Train a neural network to learn the XOR function.

In [ ]:
# XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)

print("XOR Truth Table:")
for i in range(len(X)):
    print(f"  {int(X[i][0])} XOR {int(X[i][1])} = {int(y[i][0])}")

In [ ]:
# Build XOR model
xor_model = models.Sequential([
    layers.Dense(4, activation="relu", input_shape=(2,)),
    layers.Dense(1, activation="sigmoid"),
])

xor_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

xor_model.summary()

In [ ]:
# Train the model
print("Training XOR classifier...")
history = xor_model.fit(X, y, epochs=500, verbose=0)

print(f"Initial loss: {history.history['loss'][0]:.4f}")
print(f"Final loss: {history.history['loss'][-1]:.4f}")
print(f"Final accuracy: {history.history['accuracy'][-1]:.4f}")

In [ ]:
# Test predictions
predictions = xor_model.predict(X, verbose=0)

print("Predictions after training:")
for i in range(len(X)):
    print(f"  {X[i].tolist()} -> {predictions[i][0]:.4f} (expected: {y[i][0]})")

## Custom Layer Example

Create a custom layer by subclassing `keras.layers.Layer`.

In [ ]:
class CustomDense(layers.Layer):
    """Custom dense layer with trainable weights."""
    
    def __init__(self, units):
        super(CustomDense, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
            name="kernel",
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias",
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Use custom layer
custom_layer = CustomDense(32)
test_input = tf.random.normal((4, 16))
output = custom_layer(test_input)

print(f"Custom layer input shape: {test_input.shape}")
print(f"Custom layer output shape: {output.shape}")
print(f"Number of trainable weights: {len(custom_layer.trainable_weights)}")

## Callbacks

Callbacks allow you to customize training behavior.

In [ ]:
# Define common callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=10,
    restore_best_weights=True,
)

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.5,
    patience=5,
)

print("Common Keras callbacks:")
print("  - EarlyStopping: Stops training when loss stops improving")
print("  - ReduceLROnPlateau: Reduces learning rate when loss plateaus")
print("  - ModelCheckpoint: Saves model during training")
print("  - TensorBoard: Logs for TensorBoard visualization")

## Model Save/Load

Different ways to save and load Keras models.

In [ ]:
# Model serialization options
print("Model saving options:")
print("  1. Full model: model.save('model.keras')")
print("  2. Weights only: model.save_weights('weights.weights.h5')")
print("  3. Architecture (JSON): model.to_json()")
print("")
print("Model loading:")
print("  1. Full model: keras.models.load_model('model.keras')")
print("  2. Weights: model.load_weights('weights.weights.h5')")
print("  3. From JSON: keras.models.model_from_json(json_string)")

In [ ]:
# Get model architecture as JSON
model_json = xor_model.to_json()
print(f"XOR model architecture (JSON preview):")
print(f"{model_json[:200]}...")

---

## Summary

In this notebook, we covered:

1. **TensorFlow Basics**: Tensors, operations, GPU detection
2. **GradientTape**: Automatic differentiation for training
3. **Keras Sequential API**: Simple layer stacking
4. **Keras Functional API**: Flexible model architectures
5. **Training**: Complete training loop with history
6. **Custom Layers**: Subclassing `keras.layers.Layer`
7. **Callbacks**: EarlyStopping, learning rate scheduling
8. **Model Persistence**: Save/load models and weights

For more information:
- [TensorFlow Tutorials](https://www.tensorflow.org/tutorials)
- [Keras Documentation](https://keras.io/guides/)
- [TensorFlow API Reference](https://www.tensorflow.org/api_docs/python/tf)